In [1]:
#import depenedencies
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
import keras
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Dropout,Flatten, Activation
from keras.layers import BatchNormalization, Reshape, Conv2DTranspose, UpSampling2D
from keras.optimizers import RMSprop
%matplotlib inline

In [ ]:
#load data
input_images = ".../quickdraw_data/apple.npy"
data = np.load(input_images)

In [ ]:
data.shape()

In [ ]:
data = data/255
data = np.reshape(data, (data.shape[0], 28, 28, 1))
img_w, img_h = data.shape[1:3]

In [ ]:
#show image as example
plt.imshow(data[4242,:,:,0], cmap='Greys')

In [ ]:
#discriminator model architecture
def build_discriminator(depth=64, p=0.4):
    
    #Define inputs
    image = Input((img_w, img_h, 1))
    
    #Convolutional layers
    conv1 = Conv2D(depth*1, 5, strides=2,
                   padding='same', activation='relu')(image)
    conv1 = Dropout(p)(conv1)

    conv2 = Conv2D(depth*2, 5, strides=2,
                   padding='same', activation='relu')(conv1)
    conv2 = Dropout(p)(conv2)

    conv3 = Conv2D(depth*4, 5, strides=2,
                   padding='same', activation='relu')(conv2)
    conv3 = Dropout(p)(conv3)

    conv4 = Conv2D(depth*8, 5, strides=2,
                   padding='same', activation='relu')(conv3)
    conv4 = Flatten()(Dropout(p)(conv4))
    
    #output layer
    prediction = Dense(1, activation='sigmoid')(conv4)
    
    #Model definition
    model = Model(inputs=image, outputs=prediction)
    
    return model

In [ ]:
#build discriminator
discriminator = build_discriminator()

In [ ]:
discriminator.compile(loss='binary_crossentropy',
                      optimizer=RMSprop(lr=0.0008,
                                        decay=6e-8,
                                        clipvalue=1.0),
                      metrics=['accuracy'])